In [ ]:
# !pip install google-cloud-aiplatform[prediction]>=1.16.0 fastapi nvtabular git+https://github.com/NVIDIA-Merlin/models.git --user

# Sklearn with Pandas - Custom Prediction Routine to get Merlin Model predictions

Your output should look like this - you are going to use the query model endpoint to create a CPR Endpoing

![](img/merlin-bucket.png)

This is similar to [the other notebook](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb) except we will be using pandas and bigquery

Topics covered
* Training sklearn locally, deploying to endpoint
* Saving data as CSV and doing batch predict from GCS
* Loading data to BQ, using BQ magics
* Running a batch prediction from BQ to BQ

In [1]:
from datetime import datetime


PROJECT = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
REGION = 'us-central1' 
BUCKET = 'gs://spotify-beam-v3'
REPOSITORY = 'merlin-spotify-cpr'
ARTIFACT_URI = f'{BUCKET}/merlin-processed'
MODEL_DIR = f'{BUCKET}/merlin-processed/query_model_merlin'
PREFIX = 'merlin-spotify'

In [2]:
# !gsutil mb -l us-central1 gs://wortz-project-bucket

In [3]:
# !echo y | docker container prune
# !echo y | docker image prune

### Set up repo and configure Docker (one-time)

In [4]:
# Create the repo if needed for the artifacts

! gcloud beta artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.beta.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [5]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


# New section - preprocessor creation.

In this section we will create a pipeline object that stores a standard scaler 
using the `PipeLine` class is important as it provides a lot of flexibility and conforms to sklearn's framework

appapp## Create a generic sklearn container that returns instances

https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_cpr.ipynb

**highly recommend reviewing this notebook first as it breaks down the custom predictor interface**

In [6]:
! rm -rf app
! mkdir app

In [16]:
%%writefile app/requirements.txt
uvicorn[standard]==0.15.0
gunicorn==20.1.0
fastapi==0.68.1
git+https://github.com/NVIDIA-Merlin/models.git
nvtabular==1.5.0
gcsfs
google-cloud-storage

Overwriting app/requirements.txt


### CPR Template from here https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines

In [17]:
%%writefile app/predictor.py

import nvtabular as nvt
import pandas as pd
import os
import json
import merlin.models.tf as mm
from nvtabular.loader.tf_utils import configure_tensorflow
configure_tensorflow()
import tensorflow as tf

class Predictor():
    """Interface of the Predictor class for Custom Prediction Routines.
    The Predictor is responsible for the ML logic for processing a prediction request.
    Specifically, the Predictor must define:
    (1) How to load all model artifacts used during prediction into memory.
    (2) The logic that should be executed at predict time.
    When using the default PredictionHandler, the Predictor will be invoked as follows:
      predictor.postprocess(predictor.predict(predictor.preprocess(prediction_input)))
    """
    def __init__(self):
        return
    
    def load(self, artifacts_uri):
        """Loads the model artifact.
        Args:
            artifacts_uri (str):
                Required. The value of the environment variable AIP_STORAGE_URI.
        """
        self._model = tf.keras.models.load_model(os.path.join(artifacts_uri, "query_model_merlin" ))
        self._workflow = nvt.Workflow.load(os.path.join(artifacts_uri, "workflow/2t-spotify-workflow"))
        return self
        
    def preprocess(self, prediction_input):
        """Preprocesses the prediction input before doing the prediction.
        Args:
            prediction_input (Any):
                Required. The prediction input that needs to be preprocessed.
        Returns:
            The preprocessed prediction input.
        """
        #handle different input types, can take a dict or list of dicts
        TEST_INSTANCE = prediction_input
        if type(TEST_INSTANCE) == list:
            pandas_instance = pd.DataFrame.from_dict(TEST_INSTANCE[0], orient='index').T
            if len(TEST_INSTANCE) > 1:
                for ti in TEST_INSTANCE[0:]:
                    pandas_instance = pandas_instance.append(pd.DataFrame.from_dict(ti, orient='index').T)
        else:
            pandas_instance = pd.DataFrame.from_dict(TEST_INSTANCE, orient='index').T
        transformed_inputs = nvt.Dataset(pandas_instance)
        transformed_instance = self._workflow.transform(transformed_inputs)
        return transformed_instance

    def predict(self, instances):
        """Performs prediction.
        Args:
            instances (Any):
                Required. The instance(s) used for performing prediction.
        Returns:
            Prediction results.
        """  
        
        loader = mm.Loader(instances, batch_size=instances.num_rows, shuffle=False)
        batch = next(iter(loader))
        return self._model(batch[0])

Overwriting app/predictor.py


In [18]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import json
import numpy as np
import os

from google.cloud import storage
from predictor import Predictor

app = FastAPI()

predictor_instance = Predictor()
loaded_predictor = predictor_instance.load(artifacts_uri = os.environ['AIP_STORAGE_URI'])

@app.get(os.environ['AIP_HEALTH_ROUTE'], status_code=200)
def health():
    return {}


@app.post(os.environ['AIP_PREDICT_ROUTE'])
async def predict(request: Request):
    body = await request.json()

    instances = body["instances"]
    # inputs = np.asarray(instances)
    preprocessed_inputs = loaded_predictor.preprocess(instances)
    outputs = loaded_predictor.predict(preprocessed_inputs)

    return {"predictions": outputs}

Overwriting app/main.py


In [19]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Overwriting app/prestart.sh


In [20]:
%%writefile app/instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Overwriting app/instances.json


In [21]:
#make it a package
!touch app/__init__.py

In [22]:
%%writefile Dockerfile

FROM nvcr.io/nvidia/merlin/merlin-tensorflow:nightly
WORKDIR /app 


COPY ./app/requirements.txt /requirements.txt
RUN pip install -r /requirements.txt
RUN pip uninstall scikit-learn -y
COPY ./app /app
EXPOSE 80

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]

Overwriting Dockerfile


In [23]:
SERVER_IMAGE = "merlin-prediction-cpr"  # @param {type:"string"} 
REMOTE_IMAGE_NAME=f"{REGION}-docker.pkg.dev/{PROJECT}/{REPOSITORY}/{SERVER_IMAGE}"

!docker build -t $REMOTE_IMAGE_NAME .

Sending build context to Docker daemon  4.441GB
Step 1/8 : FROM nvcr.io/nvidia/merlin/merlin-tensorflow:nightly
 ---> 66128dfca180
Step 2/8 : WORKDIR /app
 ---> Using cache
 ---> 2c3e54e5056f
Step 3/8 : COPY ./app/requirements.txt /requirements.txt
 ---> 6f0213955521
Step 4/8 : RUN pip install -r /requirements.txt
 ---> Running in c694bf1d7e52
  Cloning https://github.com/NVIDIA-Merlin/models.git to /tmp/pip-req-build-2035m3ia
  Running command git clone -q https://github.com/NVIDIA-Merlin/models.git /tmp/pip-req-build-2035m3ia
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Get

#### If you are debugging, be sure to set `-d` detached flag off

### Copy/paste if you want to run from console for testing
```python
docker run --gpus all -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-beam-v3/merlin-processed \
            us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr
```

##### No GPU:
```python
docker run -p 80:8080 \
            --name=merlin-prediction-cpr \
            -e AIP_HTTP_PORT=8080 \
            -e AIP_HEALTH_ROUTE=/health \
            -e AIP_PREDICT_ROUTE=/predict \
            -e AIP_STORAGE_URI=gs://spotify-beam-v3/merlin-processed \
            us-central1-docker.pkg.dev/hybrid-vertex/merlin-spotify-cpr/merlin-prediction-cpr
```

In [24]:
! docker stop $SERVER_IMAGE
! docker rm $SERVER_IMAGE
# ! docker run --gpus all -p 80:8080 \
#             --name=$SERVER_IMAGE \
#             -e AIP_HTTP_PORT=8080 \
#             -e AIP_HEALTH_ROUTE=/health \
#             -e AIP_PREDICT_ROUTE=/predict \
#             -e AIP_STORAGE_URI=$ARTIFACT_URI \
#             $REMOTE_IMAGE_NAME

merlin-prediction-cpr
merlin-prediction-cpr


In [86]:
! curl localhost/health

{}

In [25]:
## Ground truth candidate:
    # 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
    # 'artist_name_can': 'Russ', 
    # 'track_name_can': 'We Just Havent Met Yet', 
## TODO - we have to overload with candidate data because of the workflow transform, add overloaded values in the predictor
TEST_INSTANCE = {'collaborative': 'false',
                 'album_name_pl': ["There's Really A Wolf", 'Late Nights: The Album',
                       'American Teen', 'Crazy In Love', 'Pony'], 
                 'album_uri_can': 'spotify:album:5l83t3mbVgCrIe1VU9uJZR', 
                 'artist_followers_can': 4339757.0, 
                 'artist_genres_can': "'hawaiian hip hop', 'rap'", 
                 'artist_genres_pl': ["'hawaiian hip hop', 'rap'",
                       "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'",
                       "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'",
                       "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], 
                 'artist_name_can': 'Russ', 
                 'artist_name_pl': ['Russ', 'Jeremih', 'Khalid', 'Beyonc\xc3\xa9',
                       'William Singe'], 
                 'artist_pop_can': 82.0, 
                 'artist_pop_pl': [82., 80., 90., 87., 65.], 
                 'artist_uri_can': 'spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS', 
                 'artists_followers_pl': [ 4339757.,  5611842., 15046756., 30713126.,   603837.],  
                 'description_pl': '', 
                 'duration_ms_can': 237322.0, 
                 'duration_ms_songs_pl': [237506., 217200., 219080., 226400., 121739.], 
                 'n_songs_pl': 8.0, 
                 'name': 'Lit Tunes ', 
                 'num_albums_pl': 8.0, 
                 'num_artists_pl': 8.0, 
                 'track_name_can': 'We Just Havent Met Yet', 
                 'track_name_pl': ['Losin Control', 'Paradise', 'Location',
                       'Crazy In Love - Remix', 'Pony'], 
                 'track_pop_can': 57.0, 
                 'track_pop_pl': [79., 58., 83., 71., 57.],
                 'duration_ms_seed_pl': 51023.1,
                 'pid': 1,
                 'track_uri_can': 'spotify:track:0VzDv4wiuZsLsNOmfaUy2W', 
                 'track_uri_pl': ['spotify:track:4cxMGhkinTocPSVVKWIw0d',
                       'spotify:track:1wNEBPo3nsbGCZRryI832I',
                       'spotify:track:152lZdxL1OR0ZMW6KquMif',
                       'spotify:track:2f4IuijXLxYOeBncS60GUD',
                       'spotify:track:4Lj8paMFwyKTGfILLELVxt']
                     }

In [26]:
import json
json_instance = json.dumps({"instances": TEST_INSTANCE})
print(json_instance)

{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226

In [31]:
%%writefile instances.json
{"instances": {"collaborative": "false", "album_name_pl": ["There's Really A Wolf", "Late Nights: The Album", "American Teen", "Crazy In Love", "Pony"], "album_uri_can": "spotify:album:5l83t3mbVgCrIe1VU9uJZR", "artist_followers_can": 4339757.0, "artist_genres_can": "'hawaiian hip hop', 'rap'", "artist_genres_pl": ["'hawaiian hip hop', 'rap'", "'chicago rap', 'dance pop', 'pop', 'pop rap', 'r&b', 'southern hip hop', 'trap', 'urban contemporary'", "'pop', 'pop r&b'", "'dance pop', 'pop', 'r&b'", "'chill r&b', 'pop', 'pop r&b', 'r&b', 'urban contemporary'"], "artist_name_can": "Russ", "artist_name_pl": ["Russ", "Jeremih", "Khalid", "Beyonc\u00c3\u00a9", "William Singe"], "artist_pop_can": 82.0, "artist_pop_pl": [82.0, 80.0, 90.0, 87.0, 65.0], "artist_uri_can": "spotify:artist:1z7b1Pr1rSlvWRzsW3HOrS", "artists_followers_pl": [4339757.0, 5611842.0, 15046756.0, 30713126.0, 603837.0], "description_pl": "", "duration_ms_can": 237322.0, "duration_ms_songs_pl": [237506.0, 217200.0, 219080.0, 226400.0, 121739.0], "n_songs_pl": 8.0, "name": "Lit Tunes ", "num_albums_pl": 8.0, "num_artists_pl": 8.0, "track_name_can": "We Just Havent Met Yet", "track_name_pl": ["Losin Control", "Paradise", "Location", "Crazy In Love - Remix", "Pony"], "track_pop_can": 57.0, "track_pop_pl": [79.0, 58.0, 83.0, 71.0, 57.0], "duration_ms_seed_pl": 51023.1, "pid": 1, "track_uri_can": "spotify:track:0VzDv4wiuZsLsNOmfaUy2W", "track_uri_pl": ["spotify:track:4cxMGhkinTocPSVVKWIw0d", "spotify:track:1wNEBPo3nsbGCZRryI832I", "spotify:track:152lZdxL1OR0ZMW6KquMif", "spotify:track:2f4IuijXLxYOeBncS60GUD", "spotify:track:4Lj8paMFwyKTGfILLELVxt"]}}

Writing instances.json


In [28]:
! echo $json_instance > json_instance.json

In [29]:
!rm instances.json

In [32]:
! curl -X POST \
      -d @instances.json \
      -H "Content-Type: application/json; charset=utf-8" \
      localhost/predict

Internal Server Error

### Cloud build

In [ ]:
!gcloud builds submit --region={REGION} --tag=$REMOTE_IMAGE_NAME

### Create test instances 
Random selection from BQ validation table

In [ ]:
# ### push the container to registry
# !docker push $REMOTE_IMAGE_NAME

### Deploy to Vertex AI

In [ ]:
MODEL_DISPLAY_NAME = "Merlin Spotify Query Tower Model"
from google.cloud import aiplatform

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_URI,
    serving_container_image_uri=REMOTE_IMAGE_NAME
)

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

In [ ]:
endpoint.predict(instances=[TEST_INSTANCE])